# Parrot Diffusion Studies 🦜🎨

This notebook is designed to study parameters, settings and prompts against the [Stable Diffusion model](https://github.com/CompVis/stable-diffusion).

This notebook maintained by [Stephen Young](https://twitter.com/KyrickYoung) or SteveTheNinja#0616

Visit the Parrot Zone [parrotzone.art](http://www.parrotzone.art)

## Changelog
### 2.0
- paperspace port!

In [ ]:
# Create images out

from pathlib import Path
import os


try:
    from google.colab import drive
    google_drive = True
except:
    google_drive = False

if google_drive:
    ROOT_FOLDER = "generative"
    PROJECT_FOLDER = "parrot_studies"
    MOUNTED_PATH = Path('/content/drive')
    MYDRIVE_PATH = MOUNTED_PATH / "MyDrive"
    PROJECT_PATH = MYDRIVE_PATH /  ROOT_FOLDER / PROJECT_FOLDER

    IMAGES_OUT = PROJECT_PATH/ "images_out"

    drive.mount(str(MOUNTED_PATH))

    os.makedirs(PROJECT_PATH, exist_ok=True)
    os.makedirs(IMAGES_OUT, exist_ok=True)
else:
    # setup paths
    # REPLACE THIS WITH YOUR PATH
    # HINT: type $PWD in a terminal
    NOTEBOOK_PATH = Path("/notebooks")
    IMAGES_OUT = NOTEBOOK_PATH / "images_out"

    os.makedirs(IMAGES_OUT, exist_ok=True)

print("Will write output images to folder", IMAGES_OUT)

In [ ]:
#@title Setup
#  scipy ftfy 
%pip install transformers accelerate safetensors diffusers[torch]==0.19.0 invisible-watermark>=0.2.0
%pip install pydantic==1.7.4 pillow
%pip install git+https://github.com/kyrick/parrot-tools.git --no-deps

In [ ]:
from pathlib import Path
import os

from parrot_tools.utils import prepare_prompts_for_study, prepare_hybrid_prompts_for_study, read_csv
from parrot_tools.generate import run_prompts, BatchSettings

# THIS IS A DIRTY HACK TO SILENCE THE PROGRESS BAR
# THE TQDM PROGRESS BAR BOTTLENECKS CELL OUTPUT AND SLOWS THE NOTEBOOK
from tqdm.auto import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [ ]:
# Initialize the Model

from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipe.to("cuda")
    

In [ ]:
# Each count will add 1 additional style or modifier to each prompt. A positive number will flip the notebook into HYBRID MODE! :D
# Turn on hybridize_ALL_THE_THINGS to combine both styles and modifiers into the same prompts
hybrid_count = 0
hybridize_ALL_THE_THINGS = False

base_prompts = """
a portrait of a character in a scenic environment
a building in a stunning landscape
"""

# Supported formats:
# Gediminas Pranckevicius - first, last
# Aivazovsky, Ivan        - last, first (supports tabs if pasted from google sheets!)
# de Kooning, Willem      - 2+ words in the last name
# Death Burger            - a nickname of two or more words
# RHADS                   - a single nickname
styles_to_study = """
Pranckevicius, Gediminas
Aivazovsky	Ivan
RHADS
"""
# alternatively load names from a csv file
styles_csv_path = "" #"data.csv"

modifiers_to_study = """
graphic novel
vaporwave
"""
# alternatively load modifiers from a csv file
modifiers_csv_path = ""

# This text will be added to the end of every prompt
append_to_all_prompts = ""



if styles_csv_path:
    csv_styles = read_csv(styles_csv_path)
    styles_to_study += csv_styles

if modifiers_csv_path:
    csv_mods = read_csv(modifiers_csv_path)
    modifiers_to_study += csv_mods

if hybrid_count > 0:
    prompts_to_run = prepare_hybrid_prompts_for_study(base_prompts, styles_to_study, modifiers_to_study, append_to_all_prompts, hybrid_count=hybrid_count, hybridize_everything=hybridize_ALL_THE_THINGS)
else:
    prompts_to_run = prepare_prompts_for_study(base_prompts, styles_to_study, modifiers_to_study, append_to_all_prompts)

# print out the prompts to sanity check
for p in prompts_to_run:
    print(p.prompt, "-", p.base_filename)

In [ ]:
#@title Do The Run
#@markdown ## Run Settings
#@markdown set seed to -1 for random seed
batch_name = "StableXLStudy" #@param {type: "string"}
#@markdown batch_size is images per prompt
batch_size = 9 #@param {type: "integer"}
manual_seed = -1 #@param {type:"integer"}
steps = 15  #@param {type:"integer"}
cfg_scale = 12.0  #@param {type:"number"}

#@markdown set to 0 to turn off. Number of retries per NSFW result (will try with different seed)
NSFW_retries = 1 #@param {type:"integer"}

#@markdown ##Image Settings
image_ext = "jpeg" #@param ["jpeg", "png"]
width = 512  #@param {type: "integer"}
height = 512  #@param {type: "integer"}

#@markdown turn on/off dispalay of each image
display_single_images = False #@param {type:"boolean"}

#@markdown ##Grid Settings
make_batch_grid = True #@param {type:"boolean"}
grid_cols = 3 #@param {type:"integer"}
max_images_per_grid = 9 #@param {type:"integer"}
grid_padding = 0 #@param {type:"integer"}
#@markdown color name (`black`, `white`, `pink`, etc.) or hex code in format `#00004c'`
grid_bg_color = "white" #@param {type: "string"}

batch_settings = BatchSettings(
    batch_size=batch_size,
    batch_name=batch_name,
    base_path=IMAGES_OUT,
    steps=steps,
    cfg_scale=cfg_scale,
    seed=manual_seed,
    NSFW_retry=NSFW_retries,
    display_individual_images=display_single_images,
    image_ext=image_ext,
    image_w=width,
    image_h=height,
    make_grid=make_batch_grid,
    grid_cols=grid_cols,
    grid_max_images=max_images_per_grid,
    grid_padding=grid_padding,
    grid_bg_color=grid_bg_color
)

run_prompts(pipe, prompts_to_run, batch_settings)
